=================================================================================================================
# Lecture Notes: Neural Networks


##### D.Vidotto, Data Mining: JBI030 2019/2020


=================================================================================================================

Artificial Neural Networks, after decades of up and downs in the Machine Learning community, have re-gained popularity in the last 10-15 years under the name of Deep Learning. Their model structure, their flexibility, and their ability to estimate relatively difficult functions of their inputs, make them one of the most powerfool tool available to Data and Computer Scientists. Neural Networks can be specified under several different architectures that allow them to be nowaday's preferred algorithm in fields such as Computer Vision and Speech Recognition. The main downside of Neural Networks is the large number of hyperparameters that need to be tuned to make them work optimally. In this notebook, we will explore in detail Feedforward Neural Networks (also known as Multilayer Perceptron), and mention in the end other two types of architectures become famous recently: Convolutional Neural Networks and Recurrent Neural Networks. 

For this notebook, students are assumed to have followed the lecture in class and to have understood the following concepts: 
* layers and neurons
* activation functions
* Stochastic Gradient Descent
* Backpropagation
* hyperparameters of a Neural Network 

In the notebook, we will cover the following topics in more detail: 

1. Notation
1. Introduction
  * from linear models to Neural Networks
  * Digression: Biological Neural Networks
1. Components of a FNN
  * layer, neurons, activation function
  * formal model
  * adding layers to the network 
1. Setting up a FNN
  * regression
  * classification (binary and multi-class cases)
1. Training a NN with Backpropagation
  * Main Idea 
  * Stochastic Gradient Descent
      * mini-batches,epochs
      *  learning rate, momentum 
      * initialization
  * Backpropagation
      * Example: 2 layers with one neuron
      * Increasing the number of neurons
1. Setting a FNN - Other technical considerations
  * Number of layers and neurons 
  * learning rate
  * optimizer 
  * batch-size
  * activation function
  * number of epochs and early stopping 
  * $l_2$ and $l_1$ regularization
1. Other Architectures 
  * Convolutional Neural Networks 
  * Recurrent Neural Networks 
1. Other Resources

This notebook only contains the theoretical concepts underlying Neural Networks. Examples of Neural Networks implemented with Keras and scikit-learn can be found in the notebooks *Neural Networks in Practice*. 



## 1. Notation
In this notebook the following notation will be used: 
* as usual $j=1,...,p$ denotes the input feature index ($X_j$) and $i=1,...,n$ denotes the observation index ($\mathbf{x}_i$ refers to the vector of observed input for unit $i$)
* $l=1,...,L$ will denote the $l$-th (out of $L$) layers of the network, while $k=1,...,K_l$ will denote $k$-th among the $K_l$ neurons of layer $l$
* bold lowercase denotes a vector (for example $\mathbf{w}$ denotes a vector of weights; possible sub- or super-scripts will help to contextualize such vector)
* $z$ will generally denote linear combinations (plus biases) of neurons of a previous layer; for example, $z^{(1)}_k$ is the linear combination of neurons of the input layer and the weights that denote the $k$-th neuron in such layer 
* uppercase denote matrices (for example $\mathbf{W}^{(l)}$ is the matrix that stacks horizontally all the row vectors of weights of layer $l$, while $W^{(l)}_{k,m}$ refers to the $(k,m)$-th element in such matrix -it is the weight that connects neuron $m$ of layer $(l-1)$ to neuron $k$ of layer $l$; $\mathbf{W}^{(l)}_k$ is the $k$-th row vector of this matrix -the set of weights used to define the $k$-th neuron of layer $l$)
* a generic activation function is denoted with $g$, while distinct activation functions in one network are denoted with $g_1$, $g_2$, and so on. When a vector of dimension $d$ is given as input to $g$, it is implied that the output is a $d$-dimensional vector, which is the result of element-wise application of $g$: for example, if $d=2$, $g([v_1, v_2]) = [g(v_1), g(v_2)]$  
* $\sigma$ indicates the *sigmoid function*, already encountered in the logistic regression lecture 
* $\hat{y}$ denotes the *output* provided by the network (with a bit of abuse of notation, this can be both a probability or a class in classification tasks)


## 2. Introduction
**From Linear Models to Neural Networks.** Recall the function for the prediction of a linear regression model: 
$$\hat{y}_i= w_0 + \mathbf{w}^T\mathbf{x}_i = w_0 + \sum_{j=1}^{p}w_jx_{ij}$$
Graphically, a linear regression model can be represented as follows: 
<br>
<img src="./img/neural_networks/linear_regression.png" width="250"/>
<br> 

The first layer, called *input layer*, contains the *nodes* representing the features (note that a node with a value of 1 is included, to accommodate for the *bias* $w_0$), and each of the edges represent the weights of the regression model. The *output layer*, finally, contains one node reporting the prediction of the model, which in case of linear regression is a simple linear combination of the input features (plut the bias term). 

Similarly, we can represent graphically a *logistic regression* model: 
<br>
<img src="./img/neural_networks/logistic_regression.png" width="250"/>
<br> 
The difference with linear regression is that the linear combination of the features is *filtered* through the sigmoid function, which "squashes" its value between 0 and 1, transforming them into a probability (as we will see, the sigmoid function is an example of *activation function* in Neural Networks); this probability is the final output of the model. In turn, the class of the observation can be predicted as a function of this probability. 

**Neural Networks** (NN) add a level of complexity to the simple linear models, by introducing **hidden layers** (that is, unobserved sets of nodes) between the input and output layers. Each node of a hidden layer is also called **neuron**, to reflect an alleged analogy with the biological neurons. Here is a graphical example of a single-layer (i.e., having a unique hidden layer) neural network; to ease the introduction to NNs, we start by showing the case with a unique neuron (for a regression case):

<br>
<img src="./img/neural_networks/simple_nn.png" width="350"/>
<br> 

There are a couple of points that's worth stressing here: 
* the neuron $h^{(1)}_1$ denotes the first neuron (subscript) of the first layer (superscript) in the model
* the edges, once again, denote the weights (this time, this is implied in the figure)
* $h^{(1)}_1$ contains a number, which is computed in two steps: (1) first, the linar combination of the previous layer (the input layer in this case) is calculated; (2) second, such linear combination is filtered through an *activation function* (which can be the sigmoid function, but need not to) whose output is exactly $h^{(1)}_1$
* in turn, $h_1^{(1)}$ is multiplied by its weight and added to a new bias term; the result is filtered again through another activation function (in case of regression, this is simply the *identity function* which does not apply any filter) to return the final output of the network, $\hat{y}$

This type of Neural Networks is known as Feedforward Neural Network -FNN for short- as the information flows forward from the input to the output layer. The mere presence of the activation function is enough to turn the model into a non-linear one, and it makes the neuron capable of capturing the specific information located in a precise region of the feature space (we will see the activation functions more in detail in the next section). Of course, more complex networks allow capturing more complex aspects of the feature space, making in turn also the model more complex. 

Here's an example of a more complex single-layer network (with five neurons): 

<br>
<img src="./img/neural_networks/single_layer_nn.png" width="350"/>
<br> 

This new network can be described as follows: 
* the input layer is linked to each neuron in the hidden layer with different linear combinations of the input features. In practice, this means that each $h^{(1)}_k$ for $k=1,...,5$ differ from one another because of the *weights* used to create the linear combinations. This allows the neurons to focus on different aspects of the data, as each of these linear combinations is also given as input to an activation function, whose output is the value of each neuron $h^{(1)}_k$ 
* this means that we can identify 5 sets of weights used to create the five neurons: $\mathbf{w}^{(1)}_1$ is the set of weights to calculate the linear combination of neuron 1 in layer 1; and so on until $\mathbf{w}^{(1)}_5$. The transpose of these vectors (row vectors) can be stacked together to form a $5 \times p$ matrix of weights for the first layer (so that $\mathbf{W}^{(1)}_k = \mathbf{w}^{(1)^T}_k$): <br>$$\mathbf{W}^{(1)} = \begin{bmatrix} \text{---} & \mathbf{w}^{(1)T}_1 & \text{---} \\ \text{---} & \mathbf{w}^{(1)T}_2 & \text{---} \\\text{---} & \mathbf{w}^{(1)T}_3 & \text{---} \\\text{---} & \mathbf{w}^{(1)T}_4 & \text{---} \\\text{---} & \mathbf{w}^{(1)T}_5 & \text{---}\end{bmatrix}$$<br> This, of course, can be easily generalized to the case of $K_1$ neurons in the first layer
* The output node is now given by a linear combination of the neurons, which are therefore multiplied by a set of weights contained in the vector $\mathbf{w}^{(2)} = [w_1^{(2)},...,w_5^{(2)}]$ (plust a bias term); the resulting linear combination, once again, can optionally be filtered through an activation function (then again, this step is not necessary in regression problems) 

In the following figure, we can see what is the effect of manipulating the number of neurons in a single-layer FNN, on the predictions of a regression problem with a single input $X$: 

<br>
<img src="./img/neural_networks/nn_demo_neurons.png" width="800"/>
<br>

As you can see, by simply increasing the number of neurons in the single-layer FNN, the network was able to retrieve the true sinusoidal signal of the data. This is because single-layer NNs are [universal approximators](https://en.wikipedia.org/wiki/Universal_approximation_theorem): given a sufficiently large number of neurons, they can theoretically approximate any continuous function. (Be careful though: Too many neurons/layers lead to too complex models and therefore, as usual, to overfitting! This is what happens in the bottom right figure, where a network with 30 neurons was specified). 

It seems clear, therefore, that adding more neurons increases the capacity of the network to focus on different types of relationships in the data (different levels of interactions), and the activation functions allow the model to capture nonlinearities. Further flexibility is given by the fact that not only neurons, but also layers, can be added to the model; here's an example with two layers, the first one containing five neurons, and the second containing three neurons: 


<br>
<img src="./img/neural_networks/two_layer_nn.png" width="450"/>
<br>

This time we can count three sets of weights: (1) Those "linking" the input layer to the first hidden layer, with the corresponding vectors "stacked" in the matrix $\mathbf{W}^{(1)}$ as just seen. (2) Those linking the first hidden layer to the second hidden layer; similarly to what done for the first layer, we can "stack" the resulting weight vectors in the matrix $\mathbf{W}^{(2)}$; and (3), the set of weights "linking" the second hidden layer to the output layer, which can be denoted with the vector $\mathbf{w}^{(3)}$. (Note that, more in general, for a network with $L$ layers the set of weights for the output is the set number $L+1$). In total, this network includes $p\cdot5 + 5\cdot3 + 3$ weights, to be added to the 5+3+1 bias terms to obtain the total number of parameters present in the model. As we have seen so far, all the linear combinations are given to an activation function before their values are passed to the next layer. Adding this second layer further increases the flexibility of the Network, as each neuron in the second layer can describe and manipulate different types of relationships in the first hidden layer. This further increases the expressive ability of the network, reflected in the predictions given by the output layer. 

It is also very hard to interpret the output of a NN, given that the interpretation of each neuron is already very complicated with just a small number of layers. Therefore, NN's are in general mostly used for prediction.

To complete this introduction to Neural Networks, we can also see that the output layer needs not contain just one neuron; it can also be composed of several neurons. This can be useful, for example, in regression cases where we want to predict more than one output variable; or in multi-class classification (as we will see), where each output neuron corresponds to a different class of the output (here's an example for a single-layer network): 

<br>
<img src="./img/neural_networks/multi_output_nn.png" width="350"/>
<br>

All types of FNN's presented so far are also known as *dense* (or *fully connected*) *networks*, as all the nodes of a layer are connected with all nodes of the next and previous layers. These are in contrast with *sparse networks*, where some of the weights are set equal to 0, loosing in this way connections between (some of) the nodes. 


----------------------------------------------------------------------------------------------------------------------


#### Digression: Biological Neural Networks
The Artificial Neural Networks we are exploring in this notebook owe their names to the functioning of the biological neural networks of our brain; for a long time (at least until two or three decades ago) it was believed that ANN's were actually the mathematical reflection of what happens within the biological neurons of human and animal brains. Today this is no longer believed, as profound differences have been found (ANN's simplify too much what is really going on in reality). However, it can still be worth to give a small description of Biological neurons, to catch some similarities between the two paradigms. 


<br>
<img src="./img/neural_networks/biological_nn.png" width="400"/>
<br>

A biological neuron consists of a *cell body*, containing the *nucleu*s and other cell's components, as well as several branching extensions called *dendrites* and one very long extension called the *axon*. At the extremity of the axon there are the *axon terminals* (know also as *synapses*), which are connected to the dendrites or cell bodies of other neurons. The neuron produces electrical impulses, known as action *potentials of signals*; these travel along the axons and make the synapses release chemical signals (the *neurotransmitters*). If a neuron receives a sufficient amount of neurotransmitters, it is activated ("fired"), and the signal can be passed to the next neuron. This process happens within milliseconds in a network of billions of neurons. 

The analogy with ANN's resides in the fact that the information flows from "layers of neurons" to others, and an "activation function" determines whether a specific neuron is set on or off (i.e., if it is equal or not to 0) given the information (the inputs in the input layer) being processed.  
<br>

----------------------------------------------------------------------------------------------------------------------

## 3. Components of a FNN
### 3.1 Layers, Neurons, Activation Functions
As we have seen in the introductory section, the three main components of a FNN are the layers, the neurons, and an activation function (here denoted with $g$). In the following figure, we see more in detail how the information is processed in a single-layer NN (with five neurons). This is also the way in which **predictions** are performed when the model, already trained, is shown a new data point.  

<br>
<img src="./img/neural_networks/flow_nn.png" width="600"/>
<br>

1. First, the inputs are multiplied by each set of weights of the first layer, given by the rows of $\mathbf{W}^{(1)}$ (denoted with $\mathbf{W}^{(1)}_k$). For each neuron, such products are then added to each other (wighted sum of the inputs) and to the neuron-specific bias terms $(w^{(1)}_{0,1},...,w^{(1)}_{0,5})$. The results lead to the weighted sums (linear combinations) denoted with $z^{(1)}_1,...,z^{(1)}_5$ in the figure  
1. The results from the previous point are then processed through an activation function $g_1$; this can be a sigmoid (a.k.a. logistic) function, but other options are possible (as we will see in a while)
1. The result "fired" by the activation function is the value taken on by each neuron; therefore, the $k$-th neuron in the first layer can be written as $h_k^{(1)} = g_1\left(z_k^{(1)}\right) = g_1\left(w_{0,k}^{(1)} + \sum_{j=1}^{p}W^{(1)}_{k,j}x_{ij}\right)$
1. The values contained in the hidden neurons are then combined by means of a new weighted sum; that is, they are multiplied by a new vector of weights $\mathbf{w}_k^{(2)}$, and added to a new bias term $w_0^{(2)}$; the result is in the new node denoted by $z_1^{(2)}$ in the figure
1. last, such linear combination is (optionally) passed to an output activation function $g_2$ (notice that such function needs not be the same as $g_1$). If no filter is desired (as it is custom in regression problems), we can see $g_2$ as an identity function. Thus, we can write the formula for the output, $$\hat{y} = g_2\left(z^{(2)}_1\right) = g_2\left(w_0^{(2)} + \sum_{k=1}^{5}w_k^{(2)}h^{(1)}_k\right)$$ 

**Types Of Activation Functions**. The role of the activation function is to decide which bit of information is relevant for the network, when making new predictions. Such functions must be non-linear in nature; otherwise, the network becomes a combination of linear models, which at last is again a linear model! What makes Neural Networks universal approximators, instead, is the presence of non-linear activation functions. 

Ideally, the activation functions (at least the ones that define the hidden layers) should somehow compress (*squash*) the value of the linear combinations of their inputs into a bounded range of values, or at least to set some of these linear combinations to 0, in which case the neurons are switched off and not used to predict the current unit. For any type of NN architecture, there are some standard activation functions that are commonly used. 

Until few years ago, the most common activation function was the **sigmoid function**; as we have already described above and in the lecture of logistic regression, this function takes an input and transforms it into a value between 0 and 1. This means that when the sigmoid function is used, the values of the neurons cannot be smaller than 0 and larger than 1. For a linear combination (plus bias) of the previous layer $z$, the sigmoid function is defined as: 

$$ g = \sigma(z) = \frac{1}{1+e^{-z}}.  $$

The bias terms can then be seen as the magnitude that the linear combination of the inputs must take to have a neuron equal to 0.5. 

Another common function is the **tanh** (hyperbolic tangent) function; it is S-shaped, similar to the sigmoid function, but instead of 0 and 1, it ranges between -1 and 1 (in which case the neurons can also be negative, bounded at -1). In fact, the tanh function can be seen as a rescaled sigmoid. In this case, the bias tells the minimum value needed for the linear combination in order to yield a neuron equal or larger than 0. The tanh function is defined as: 

$$ g = tanh(z) = \frac{e^{z} - e^{-z}}{e^{z} + e^{-z}} $$

A last function that has gained popularity in the last years, and that has become the default choice in several Deep Learning libraries, is the **Rectified Linear Unit** (ReLU). It switches off to 0 all the inputs smaller than or equal to 0, while if the input is positive, it simply becomes the identity function (and therefore linear in the input). However, the ReLU function is a non-linear function, as it only activates one part of its input. It is then defined as: 

$$
g = ReLU(z) =
\left\{
\begin{array}{cl}
0 & if\ z \leq 0 \\
z & if\ z > 0 
\end{array}
\right.
$$
Therefore, ReLU switches on only those neurons for which the linear combination of its inputs is larger than the bias term. The reason why the rectified linear unit has become so popular is that it works well in practice, and makes the gradients for algorithms such as gradient descent cheap to compute. 

The following graph plots the three functions just discussed: 

<br>
<img src="./img/neural_networks/activation_functions.png" width="400"/>
<br>


A nice overview and more detailed discussion of these and other activation functions (such as for example the leaky ReLU) is given in [this link](https://www.analyticsvidhya.com/blog/2020/01/fundamentals-deep-learning-activation-functions-when-to-use-them/).


### 3.2 Formal model 
We repeat now the steps seen previously, but using exclusively a mathematical notation. 

1. The input values of the $i$-th unit are multiplied by the $k$-th set of weights and added to the $k$-th bias term: $z^{(1)}_k=w_{0,k}^{(1)} + \mathbf{W}^{(1)T}_k\mathbf{x}_i$
1. The $k$-th neuron in the hidden layer is calculated by passing the output of the previous point into the activation function: $h^{(1)}_k = g_1\left(z_k^{(1)}\right)$
1. Finally, the output of the network for unit $i$ is computed by repeating the previous two steps using layer 1 as input: $\hat{y}=g_2\left(z^{(2)}_1\right) =g_2\left(w^{(2)}_{0} + \mathbf{w}^{(2)T}\mathbf{h}^{(1)}\right)$, where $\mathbf{h}^{(1)}$ is the vector that contains all the values of the neurons of layer 1 

Note that we can also rewrite steps (1) and (2) as $\mathbf{h}^{(1)} = g_1\left(\mathbf{w}^{(1)}_0 + \mathbf{W}^{(1)}\mathbf{x}_i\right)$ with $\mathbf{w}^{(1)}_0$ the vector containing all the bias terms for the first layer. 

The following table summarizes the computations for each layers. 

| Input Layer | Hidden Layer ${h}^{(1)}$ | Output Layer $\hat{y}$ | 
|:-----------:|:-----------:|:-----------:|
|$\mathbf{x}_i$| $$g_1\left( \mathbf{w}^{(1)}_0 + \mathbf{W}^{(1)}\mathbf{x}_i\right)$$ | $$g_2\left(w^{(2)}_{0} + \mathbf{w}^{(2)T}\mathbf{h}^{(1)}\right)$$ | 

Notice, last, that by "wrapping" the first layer inside the output layer, the output can be written as a function of the input: 

$$\hat{y} = g_2\left(w^{(2)}_{0} + \mathbf{w}^{(2)T}\left[g_1\left( \mathbf{w}^{(1)}_0 + \mathbf{W}^{(1)}\mathbf{x}_i\right)\right]\right)$$


### 3.3 Adding Layers to the Network
When layers are added to the network, the principle behind the computations seen so far is the same of the single-layer case: that is, we compute as many linear combinations of the neurons in the previous layers as there are neurons in the next layer, filter them through the activation function, and go to the next layer. The following table shows an example with two layers; $\mathbf{W}^{(2)}$ and $\mathbf{w}^{(2)}_0$, represent the matrix of weights and vector of biases for the second hidden layer, while $\mathbf{w}^{(3)}$ and $\mathbf{w}^{(3)}_0$ represent the vector of weights and the bias of the output layer. It is assumed that for the two hidden layers, the same activation function $g_1$ is used (this is not necessary though). The output layer, instead, uses a different function $g_2$ (the reason of this difference will become clearer in the next section).  


| Input Layer | Hidden Layer 1: ${h}^{(1)}$ | Hidden Layer 2: ${h}^{(2)}$ | Output Layer $\hat{y}$ | 
|:-----------:|:-----------:|:-----------:|:-----------:|
|$\mathbf{x}_i$| $$g_1\left( \mathbf{w}^{(1)}_0 + \mathbf{W}^{(1)}\mathbf{x}_i\right)$$ |$$g_1\left( \mathbf{w}^{(2)}_0 + \mathbf{W}^{(2)}\mathbf{h}^{(1)}\right)$$ | $$g_2\left(w^{(3)}_{0} + \mathbf{w}^{(3)T}\mathbf{h}^{(2)}\right)$$ | 

And writing the output as a function of the input: 


$$\hat{y} = g_2\left(w^{(3)}_{0} + \mathbf{w}^{(3)T}\left\{g_1\left( \mathbf{w}^{(2)}_0 + \mathbf{W}^{(2)}\left[g_1\left( \mathbf{w}^{(1)}_0 + \mathbf{W}^{(1)}\mathbf{x}_i\right)\right]\right)\right\}\right)$$


More in general, in a network with $L$ layers, the $l$-th layer can be expressed as a function of the previous ($l-1$) layer... 

$$g_1\left(\mathbf{w}^{(l)}_0 + \mathbf{W}^{(l)}\mathbf{h}^{(l-1)}\right)$$

...and in turn it becomes the input of the next ($l+1$) layer. The output layer, in a network with $L$ layers, is a function of the ($L+1$)-th sets of weights and biases. If the output layer is also composed of multiple neurons (we have seen an example in the introduction), then the ($L+1$)-th set of weights becomes a matrix (with $K_{L+1}$ rows, where $K_{L+1}$ is the number of neurons in the output layer), denoted by $\mathbf{W}^{(L+1)}$. The biases of the output, in turn, become the components of a $K_{L+1}$-dimensional vector, $\mathbf{w}^{(L+1)}_0$.   

## 4. Setting Up a Neural Network
Finding a good setting of a NN is not easy, as it requires tuning several hyperparameters (more on this in Section 6 below). Nevertheless, depending on whether we are in a regression or in a (binary/multi-class) classification problem, there are some standard rules that can be followed when deciding on the architecture of the network. In particular, depending on the type of prediction we want to perform, such architectures differ for the number of neurons in the output layer, the activation function for the output layer, and the **loss function** used to find the optimal weights of the network during the training stage (this is done with an algorithm called **backpropagation**, as we will see in Section 5). In this section, we will discuss how to set such components; in Section 6, we will come back to the problem of setting the number of neurons and layers and the activation functions in the hidden layers. 

### 4.1 Regression 
In regression, the typical architecture is the one we have discuss the most so far. That is, we only need one neuron on the output layer, and the identity function ($g_2(z) = z $) can be used as activation of the output. For the loss function $J$ to be used during optimization (this function is denoted with $L$ in other notebooks), a typical choice is given by the mean squared error: 

$$J = MSE(y, \hat{y}) = \frac{1}{n}\sum_{i=1}^n(y_i-\hat{y}_i)^2$$. 

There can be some exceptions to these rules. For example, if we want to constrain the output to be positive, we can use the ReLU as the activation function for the output. Otherwise, if we want to bound the output to lie within the (0-1) interval, we can use the sigmoid function; if we want to restrict it into the (-1,1) interval, we can use the tanh function. Last, if we want to predict more than one target $y$, we can specify as many neurons as the number of targets for the output layer. 

### 4.2 Classification
**Binary Classification**. In binary classification, we can arbitrarily set the output labels equal to 0 (negative class) or 1 (positive class). Here, we also use one neuron for the output layer. Such neurons denote the probability $\Pr(y)$ of the positive class. Once such probability is computed, the class can be predicted based on a threshold (usually 0.5).   

<br>
<img src="./img/neural_networks/binary_nn.png" width="300"/>
<br>

Therefore, in order to express the output as a probability, the activation function $g_2$ for the output layer is the sigmoid function. The loss function, instead, is the [*cross-entropy*](https://en.wikipedia.org/wiki/Cross_entropy); this was already introduced in the multi-class case of logistic regression, and we repropose it here ($\hat{y}$ represent the predicted probability of the positive class): 

$$J = H(y,\hat{y}) = -\frac{1}{n}\sum_{i=1}^{n}\left(y_i log(\hat{y}_i)+(1-y_i)log(1-\hat{y}_i)\right)$$

Notice that this function is minimized (equal to 0) when $y_i=\hat{y}_i$ (remember: $y_i$ can be equal to either 0 or 1), and maximal when predictions and labels do not match. Thus, since it measures the discrepancies between predictions and output, it is a valid loss (cost) function. Alternatively, the mean squared error MSE can also be used in binary classification (in which case the squared difference between the value of the observed class and its estimated probability is considered). 


**Multi-Class Classification**. Here, we have multiple classes, so that $y_i$ can take on $C$ possible values, from 1 to C. In this case, it is customary to one-hot encode the labels; for example, if for the $i$-th unit we observe $y_i=c$, its output will be converted to a vector $y_i=[0,0,...,1,...,0]$, where the '1' is in the position of the $c$-th class. At this point, the network is set with as many neurons in the output layers as the number of classes ($C$). Each value predicted by the output $\hat{y}_1,...,\hat{y}_C$ can be considered as the probability of class $c$; thus, in this case, the activation function $g_2$ for the output is the *softmax* function. We also encountered this function when discussing the multi-class case of logistic regression. If we denote with $z_c$ the value $w^{(L+1)}_{0,c} + \mathbf{W}^{(L+1)}_c\mathbf{h}^{(L)}$ (where, as usual, $w^{(L+1)}_{0,c}$ is the $c$-th bias for the output layer, $\mathbf{W}^{(L+1)}_c$ is the $c$-th row of the weights matrix for the output layer, and $\mathbf{h}^{(L)}$ are the neurons of the last hidden layer), the $c$-th neuron in the output layer is then estimated as: 

$$ softmax(z_c) = \frac{exp(z_c)}{\sum_{c'=1}^{C} exp(z_{c'})}. $$

In practice, the softmax function is the multi-class version of the sigmoid, and calculates the probabilities for each of the output neurons. Once again, the loss function is the cross-entropy, extended to the case with $C$ classes:   

$$J = H(y,\hat{y}) = -\frac{1}{n}\sum_{i=1}^{n}\sum_{c=1}^{C}\left(y_{i,c} log(\hat{y}_{i,c})\right).$$ 

Alternatively, like in the binary case, the MSE can be used also for the multi-class case (in which case the interpretation of the error is analogous to the one described for the binary case; the difference is that the MSE is now calculated for each node in the output layer). 

**Summary.** The following table summarizes the standard choices for the architectures of NN's in the three cases just outlined. 

|     | Regression | Binary Class. | Multi-Class Class.|
|:---:|:---:|:---:|:---:|
| **# Output Neurons** | 1 | 1 | $C$ | 
| **Output Layer Activation** | None (Identity) | Logistic | Softmax | 
| **Loss Function** | MSE | Cross-Entropy | Cross-Entropy | 


## 5. Training a NN with Backpropagation
### 5.1 Main Idea
Training a NN network consists of finding optimal values for the weights and biases, in such a way that the loss functions described in the previous section reach a minimum. Like several algorithms studied in this course, NN also use Gradient Descent to iteratively estimate the parameters. If we store all the weights and biases of the model in a *tensor* (i.e., a multi-dimensional array) $\mathbf{W}$, we can describe the typical Gradient Descent updating step: 

$$\mathbf{W}_{new} \leftarrow \mathbf{W}_{old} - \eta \nabla J(\mathbf{W}_{old})$$

where $J(\cdot)$ corresponds to any of the loss functions describred in the previous section, expressed as a function of the parameters of the network. This version of Gradient Descent, in which we use all $n$ observations in the training set to perform an update step, is called *batch gradient descent*. 

The optimization problem to solve when training a NN is highly non-convex (the number of parameters to optimize grows quickly as we add few neurons to the network), and the function to optimize will contain several local minima, making it impossible to find the global optimum. In practice, what is done is to accept the convergence to a local minimum, as long it is a wide (not necessarily deep) point in the parameter space that allows for good generalization of the model predictions (these types of regions generally do not overfit the training dataset). Batch gradient descent, under this point of view, is problematic: besides being slow to converge (especially with large training datasets), it might easily get stuck in some narrow local minimum which might be too dataset-specific, jeopardizing in this way model performance. For this reason, in the Deep Learning community, a special version of Gradient Descent, called *Stochastic* (or mini-batch) *Gradient Descent*, has become popular. 

### 5.2 Stochastic Gradient Descent
**Mini-Batches and Epochs**. In mini-batch Gradient Descent, instead of calculating the gradient of the cost function on the whole training dataset before performing the update, the training data is randomly split into *mini-batches* (these are simply subsets of units of the training dataset), and the gradient is iteratively evaluated for each of them. The update of the model parameters is executed after a mini-batch has been evaluated, rather than the whole training dataset. One iteration of the mini-batch GD algorithm, (which occurs after all mini-batches have been evaluated so that the algorithm has passed once through the whole training dataset) is called **epoch**. The size of the mini-batches (called "batch size"), $n_b$, is another hyperparameter that can be tuned by the user; typical choices for $n_b$ are 32, 64, or 128, as in this way it is possible to exploit CPU's and GPU's capacity to perform parallel matrix and vector multiplication. When using mini-batch GD, the loss functions that are used are still the ones described in Section 4, with the difference that they are calculated for each batch rather than for the whole dataset (therefore, $n$ in the loss functions should be replaced by $n_b$). In the special case of $n_b=1$, the algorithm is called "Stochastic Gradient Descent", SGD (although in general, libraries and packages refer to SGD also when using mini-batch GD).

In this figure, you can see the different paths taken from batch GD, mini-batch GD, and stochastic GD when optimizing a two-parameter function: 

<br>
<img src="./img/neural_networks/gradient_descent_convergence.png" width="400"/>
<br>

As you can see, batch GD reaches the minimum in a more stable and efficient way, while SGD is much noisier and when it gets close to the minimum it keeps "dancing" around it, without reaching it exactly. Mini-batch GD, instead, is somewhat in the middle, being noisier that batch GD but more stable than SGD. However, the advantage of mini-batch methods (including SGD) is that they require much less data to update the parameters of the network; such updates, therefore, occur more often than in standard GD. Furthermore, given to the noise they introduce, they are more likely to escape from a local minimum in a narrow "hole", allowing to conduct the algorithm to a better region of the parameter space (in general a wider local minimum). In contrast, batch GD does not have this property, and it is "doomed" to converge to the first local minimum it finds, no matter how "deep" it is. 

**Learning Rate and Momentum**. Among the disadvantages of SGD and mini-batch GD, is that there is a new hyperparameter to be tuned ($n_b$), and that they are more sensitive to the **learning rate** $\eta$ for the update step. A way to overcome this issue is by making the learning rate function of the iteration step $t$, so that it changes at every iteration. What is done usually is to start with a large value of $t$ when the algorithm is far from the minima, and decreasing it according to a selected "schedule" as the number of iteration $t$ grows, and the algorithm gets closer to a minima, so that it becomes more precise. Example of schedules are *power scheduling* ($\eta(t) = \eta_0/(1+t/s)^r$, with $t/s$ decay parameter defined by the iteration number $t$ and a hyperparameter $s$, and $\eta_0$ the starting value of $\eta$; $r$ is another hyperparameter), and *exponential scheduling* (with $\eta(t) = \eta_00.1^{t/s}$). Of course, other schedules are possible. 

Another way to accelerate learning is by using the *momentum*; momentum is a way to take into account the directions indicated by the gradients in the previous iterations, so that the algorithm does not depend only on the direction of the last iteration and robustly moves towards the compund main direction indicated by previous steps. This can be beneficial in presence of long curvatures in the objective function, or noisy gradients. Convergence is accelerated and the algorithm is more likely to escape bad local optima. There are two update steps in GD with momentum: 
$$1.\ \mathbf{m}_{new} \leftarrow \beta\mathbf{m}_{old} - \eta\nabla J(\mathbf{W}_{old})$$
$$2.\ \mathbf{W}_{new} \leftarrow \mathbf{W}_{old} + \mathbf{m}_{new}$$

Here, $\mathbf{m}$ is the momentum vector (where all old gradients are stored), while $\beta \in [0,1)$ is the *momentum* hyperparameter. The larger $\beta$, the more previous gradients affect the direction of the new update. Typical values of $\beta$ are 0.9 and 0.99. The learning rate now depends on $\beta$; GD with momentum can be $\frac{1}{1-\beta}$ times faster than GD without momentum. With $\beta = 0.9$, momentum can become 10 times faster than GD methods without it. 

The following figure represents a possible sequence of GD iterations, with and without momentum. Note how the steps are more "stable" in presence of momentum, and how the algorithm requires less iterations to get closer to the minimum.  

<br>
<img src="./img/neural_networks/momentum.png" width="400"/>
<br>

**Data Scaling**. Although not strictly necessary for the neural network model, it is a good idea to scale the continuous features of the dataset before training. This can help to speed up the convergence of Gradient Descent. In fact, with rescaled features, the cost function looks more symmetric, while with un-scaled inputs the cost function might be more elongated, and squished towards some specific direction. The latter case is known to create convergence problems to GD methods, and a smaller learning rate might be required to reach the local minimum. Conversely, in a nicely symmetric cost function all directions share the same type of curvature, and GD can potentially find the direction of a a good local minimum with the help of a larger learning rate and - therefore - within a smaller number of iterations.

**Initialization**. In order to perform any type of GD, all weights and biases need to be initialized. While there are some ways to initialize these values that help to speed up convergence, for us it is just important to know the heuristics that the weights can be initialized with draws from a random distribution (it can be a uniform or Gaussian), set in such a way that the values are very small and possibly close to 0. This avoids obtaining very large values of the weigths, and consequently exploding values for the neurons in the first iterations, which would result in too unstable results. Furthermore, too large weights cause activation functions such as the sigmoid or tanh to saturate (i.e., they reach their extreme values) in regions where the function is flat, and therefore the gradient is basically 0 (which prevents learning). 


### 5.3 Backpropagation
Backpropagation is a way to compute the gradients of a neural network model, so that it can be exploited by GD methods. Because, as we have seen in Section 3, the outputs of a Neural Network are expressed in terms of *function compositions* ($h(x) = g(f(x))\ $), backpropagation makes intensive use of the [chain rule](https://en.wikipedia.org/wiki/Chain_rule) to compute the gradients. 

In this section we will see the functioning of Backpropagation with a stochastic gradient descent step ($n_b=1$) to ease the presentation. The reasoning can be easily extended to the $n_b>1$ case. In particular, Backpropagation with SGs works as follows: 

1. consider the next unit $i$ of the dataset, and given its inputs perform a feed-forward step (as seen in Section 3) in such a way that you can calculate the loss for that unit 
1. using the chain rule to compute the gradient of such loss, w.r.t. the weights in the last hidden layer $\mathbf{h}^{(L)}$ 
1. next, compute the gradient of such loss w.r.t the weigths of the last-but-one hidden layer $\mathbf{h}^{(L-1)}$ (once again with the chain rule)
1. Continue until the gradient w.r.t. the weights of all hidden layers (from $L$ to $1$) are computed
1. Perform the SGD update with the gradient just found 
1. Repeat steps 1-4 until convergence

"Backpropagation" owes its name to the fact that, after the forward step 1 which goes from the input to the output layers, the error is propagated back from the last to the first hidden layers, in order to obtain the gradient values.

As done above, we will refer to a linear combination of nodes (plus bias) with $z$. Therefore, from the input layer to the first hidden layer, $z^{(1)}_k$ refers to $w^{(1)}_{0,k} + \mathbf{W}^{(1)}_k\mathbf{x}_i$, while for the second hidden layer, $z^{(2)}_k = w^{(2)}_{0,k} + \mathbf{W}^{(2)}_k\mathbf{h}^{(1)}$. The $k$-th neuron in the first layer is then $h^{(1)}_k = g_1\left(z^{(1)}_k\right)$, with $g_1$ the chosen activation for the hidden layers. Similarly, the $k$-th neuron in the second hidden layer can be expressed as $h^{(2)}_k = g_1\left(z^{(2)}_k\right)$. The activation function chosen for the output layer will be denoted by $g_2$, and the $k$-th neuron of the output layer in a two-layers network is $\hat{y} = g_2\left(z^{(3)}_k\right)$.   

#### Example: two layers with one neuron 
Here, we see how backpropagation works in a simple case, with two hidden layers and one neuron per layer (in the next section we extend the network to more neurons). The network is represented in this figure, where weights and the linear combinations $z$ are also reported for a better understanding of the algorithm: 


<br>
<img src="./img/neural_networks/backprop_1.png" width="700"/>
<br>

We consider an example with the squared error as loss function. As we consider only one unit at a time, the loss for each iteration of SGD (divided by two to ease the computation with the derivatives) is: 

$$J = \frac{1}{2}(y_i - \hat{y}_i)^2.$$

The loss can be rewritten as a function of the elements in the second hidden layer: 

$$ J =  \frac{1}{2}\left(y_i - g_2(z^{(3)}_1)\right)^2 = \frac{1}{2}\left(y_i - g_2(w_0^{(3)} + w_1^{(3)}h_1^{(2)})\right)^2. $$

Now, let's assume we have already performed the first feed-forward step, and the loss has been calculated. Backpropagation starts by finding the derivative of the loss w.r.t. $w_0^{(3)}$ and $w_1^{(3)}$.  With the help of the chain rule:  
<br>
$$\frac{\partial J}{\partial w_1^{(3)}} = \frac{\partial J}{\partial \hat{y}}
\frac{\partial \hat{y}}{\partial z_1^{(3)}}\frac{\partial z_1^{(3)}}{\partial w^{(3)}_1} =
\left(\hat{y}_i-y_i\right)\cdot g_2'(z_1^{(3)}) \cdot h^{(2)}_1 
$$
<br>


where $g_2'(z_1^{(2)})$ is the derivative of the activation function of the output layer (for example, if $g_2$ is the sigmoid function $\sigma(z)$, its [derivative](https://towardsdatascience.com/derivative-of-the-sigmoid-function-536880cf918e) is $\sigma(z)(1-\sigma(z))$). In a similar fashion, we can find the partial derivative w.r.t. $w_0^{(3)}$: 

$$\frac{\partial J}{\partial w_0^{(3)}} = \frac{\partial J}{\partial \hat{y}}
\frac{\partial \hat{y}}{\partial z_1^{(3)}}\frac{\partial z_1^{(3)}}{\partial w^{(2)}_0} = 
\left(\hat{y}_i-y_i\right)\cdot g_2'(z_1^{(3)}) \cdot 1
$$
<br>
**Note**. If we use the identity function for $g_2$ (so that no activation is used for the output), $$\frac{\partial \hat{y}}{\partial z_1^{(3)}} = 1$$ 
<br>
We have found the formulas to compute the first two components of our gradient vector. Let's continue the back-propagation, and find the derivatives w.r.t. $w^{(2)}_1$ and $w^{(2)}_0$. In order to do this, we compute the derivative of the loss w.r.t. the hidden neuron $h^{(2)}_1$, as it will simplify computations in the subsequent steps: 

<br>
$$\frac{\partial J}{\partial h_1^{(2)}} = 
\frac{\partial J}{\partial \hat{y}}
\frac{\partial \hat{y}}{\partial z_1^{(3)}}
\frac{\partial z_1^{(3)}}{\partial h^{(2)}_1} = 
\left(\hat{y}_i-y_i\right)\cdot g_2'(z_1^{(3)}) \cdot w^{(3)}_1 
$$
<br>

The first two components of this last derivative, $\frac{\partial J}{\partial \hat{y}}$ and $\frac{\partial \hat{y}}{\partial z_1^{(3)}}$, were already computed in the previous step, and therefore $\frac{\partial J}{\partial h_1^{(2)}}$ can be calculated efficiently. Now, the derivative w.r.t. $w^{(2)}_1$ is: 

<br>
$$\frac{\partial J}{\partial w_1^{(2)}} = 
\frac{\partial J}{\partial \hat{y}}
\frac{\partial \hat{y}}{\partial z_1^{(3)}}
\frac{\partial z_1^{(3)}}{\partial h^{(2)}_1} 
\frac{\partial h^{(2)}_1}{\partial z^{(2)}_1} 
\frac{\partial z^{(2)}_1}{\partial w^{(2)}_1}
$$
<br>
This expression looks quite formidable, but if you pay a bit more attention to it, we can see that most of its components were already calculated in previous steps. In fact, this can rewritten as: 

<br>
$$\frac{\partial J}{\partial w_1^{(2)}} = 
\frac{\partial J}{\partial h_1^{(2)}} 
\frac{\partial h^{(2)}_1}{\partial z^{(2)}_1} 
\frac{\partial z^{(2)}_1}{\partial w^{(2)}_1} = 
\frac{\partial J}{\partial h_1^{(2)}} \cdot g_1'(z_1^{(2)}) \cdot h^{1}_1
$$
<br>
where $\frac{\partial J}{\partial h_1^{(2)}}$ was precisely computed in the previous step, and $g_1'$ is the derivative of the activation function for the hidden layers. Similarly, the gradient w.r.t. $w_0^{(2)}$ is: 
<br>
$$\frac{\partial J}{\partial w_0^{(2)}} = 
\frac{\partial J}{\partial h_1^{(2)}} 
\frac{\partial h^{(2)}_1}{\partial z^{(2)}_1} 
\frac{\partial z^{(2)}_1}{\partial w^{(2)}_0} = 
\frac{\partial J}{\partial h_1^{(2)}} \cdot g_1'(z_1^{(2)}) \cdot 1
$$
<br>
By now, you should have noticed what pattern is going on here: first, the derivatives w.r.t. the neurons of previous layers are computed and stored; second, the derivatives w.r.t. the weights of such layers are then calculated. With this rule in mind, it is straightforward to find the gradients for the bias and weights that connect the input and first hidden layer. First, the derivative w.r.t. the first hidden layer is: 

<br>
$$\frac{\partial J}{\partial h_1^{(1)}} = 
\frac{\partial J}{\partial h_1^{(2)}} 
\frac{\partial h^{(2)}_1}{\partial z^{(2)}_1} 
\frac{\partial z^{(2)}_1}{\partial h^{(1)}_1} = 
\frac{\partial J}{\partial h_1^{(2)}} \cdot g_1'(z_1^{(2)}) \cdot w^{(2)}_1
$$
<br>
and, in turn, the partial derivatives w.r.t. the $j$-th weight for the first hidden layer: 
<br>
$$\frac{\partial J}{\partial w_{1,j}^{(1)}} = 
\frac{\partial J}{\partial h_1^{(1)}} 
\frac{\partial h^{(1)}_1}{\partial z^{(1)}_1} 
\frac{\partial z^{(1)}_1}{\partial w^{(1)}_{1,j}} = 
\frac{\partial J}{\partial h_1^{(1)}} \cdot g_1'(z_1^{(1)}) \cdot x_{ij}
$$
<br>
for $j=1,...,p$. The gradient w.r.t. the last bias term is: 
<br>
$$\frac{\partial J}{\partial w_0^{(1)}} = 
\frac{\partial J}{\partial h_1^{(1)}} 
\frac{\partial h^{(1)}_1}{\partial z^{(1)}_1} 
\frac{\partial z^{(1)}_1}{\partial w^{(1)}_0} = 
\frac{\partial J}{\partial h_1^{(1)}} \cdot g_1'(z_1^{(1)}) \cdot 1
$$
<br>
And that's it! We have the formulas for the gradients of all the weights in the model. As you can see, by storing the information w.r.t. neurons and weights in upper levels of the networks, backpropagation allows to efficiently calculate the derivatives w.r.t. neurons and weights in lower layers. Now, it is just a matter of plugging the values found for the gradient in the parameter update step, and the full SGD iteration is complete. 

#### Increasing the number of neurons
When multiple neurons are present in the layers, the principle seen in the previous section is the same. Let's suppose that a generic layer $l$ (in a network with $L$ layers in total) has $K_l$ neurons, while the layer $l+1$ in the next level (already explored by backpropagation) has $K_{l+1}$ neurons. There are only a couple of differences with the algorithm seen above. First, the gradient for neuron $h^{(l)}_k$ (with $k\ \in \{1,...,K_l\}$) is now calculated for each neuron at the level above, and its gradient is taken to be the sum over all such neurons (if $l=L$, level $l+1$ corresponds to the output, $\hat{y}$): 

$$\frac{\partial J}{\partial h_k^{(l)}} = 
\sum_{m=1}^{K_{l+1}}\frac{\partial J}{\partial h_m^{(l+1)}} 
\frac{\partial h_m^{(l+1)}}{\partial z_m^{(l+1)}} 
\frac{\partial z_m^{(l+1)}}{\partial h_k^{(l)}} 
$$

and a generic weight $W^{(l)}_{k,r}$ which connects neuron $r$ of layer $(l-1)$ with neuron $k$ of layer $l$ (the element $(k,r)$ of the weights matrix $\mathbf{W}^{(l)}$): 

$$
\frac{\partial J}{\partial W_{k,r}^{(l)}} = 
\frac{\partial J}{\partial h_k^{(l)}}
\frac{\partial h_k^{(l)}}{\partial z_k^{(l)}}
\frac{\partial z_k^{(l)}}{\partial W_{k,r}^{(l)}}.
$$

When multiple neurons are  present in the output layer, the principle is the same: we compute the gradients for each of the output neurons, and perform the sum over such neurons.  


## 6. Setting a FNN - Other technical considerations
NN are among the most difficult algorithms to tune, as they are very sensitive to the values of their (several) hyperparameters. In this section, we see more in detail other practices, advices, and tricks to keep in mind when setting and training a FNN. In general, it is possible to use Cross-Validation to tune a Neural Network; however, this can be a time-consuming step, and what is done in practice (especially if the size of the dataset is very large) is to use a validation set to check the network performance after (and during!) training. 

**Number of layers and neurons**. Since single-layer networks are universal approximators, in most of the problems shallow networks (i.e., networks with a small number of layers) should be able to perform well. However ,despite the fact that single-layer networks can approximate any function, they might need an exponentially growing width (=number of neurons) to reach optimal performance. On the other hand, deeper networks that are less wide can be more efficient to compute and get closer to optimal performance. In practice, one or two layers should be able to solve most (90%) of the problems; if optimal performance is not reached yet, you can try to make the network deeper (by gradually increasing the number of layers). 

About the number of neurons, a common structure given to FNN's (especially in the past) is a "pyramid", in which the number of neurons is decreased at each layer. With the newly developed methods to regularize NN's, more recent networks are built with the same number of neurons at each hidden layer. This also allows decreasing the number of hyperparameters to tune. As with the number of layers, you can also start with a small number of neurons, and try to increment it to see whether performance improves. Another rule of thumb is to use a number of neurons for the hidden layers somewhere in between the number of neurons in the input layer (i.e., the number of features) and the number of neurons in the output layer (for example, by using a mean among these two numbers).    

Of course, both the deep and the width of the networks must be carefully chosen; too little layers and neurons cause underfitting, while too deep and wide networks easily overfit your dataset. Furthermore, adding just a few neurons or layers to the network grows quickly the number of parameters present in the network, slowing down algorithms such as Gradient Descent. You can find a good discussion on the number of layers and neurons to use in a FNN in this [stackoverflow page](https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw).

**Learning Rate**. Gradient Descent methods are very sensitive to the value chosen for the learning rate. In general, there is a tradeoff between the speed of the algorithm and the precision of the final results, with larger learning rates getting close to a local minimum faster, but with the risk of never converging, and smaller rates that may take too long time to converge. We have already discussed in the previous section possible learning schedules, as well as the momentum technique, that allow the learning rate to adapt and change at each iteration. Alternatively, a rule of thumb may be to start with a very small learnign rate (e.g., $10^{-5}$) and increase it gradually to check how model performance changes with it. When the learning rate becomes too large, generalization performance should drop. A good discussion on possible strategies to set the learning rate and other methods is given in the blog [machinelarningmastery.com](https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/).   

**Optimizers**. Besides (batch/mini-batch/stochastic) Gradient Descent methods, a number of other optimizer has been developed which is suitable for neural nets. Choosing a good optimizer may not only speed up computations, but also ensures to reach a local minimum that generalizes well to new data, in a more efficient manner. While a nice overview of some of these main methods is reported in this [medium blog](https://medium.com/datadriveninvestor/overview-of-different-optimizers-for-neural-networks-e0ed119440c3), here it is worth mentioning the ADAM optimizer. ADAM (that stands for *adaptive moment estimation*) uses a weighted average between previous and current iterations of the first and second moments of the gradients (the first moment is the mean and the second moment is the uncentered variance). In this way, ADAM does not only exploits the benefits of the momentum method (first moment), but it also uses an adaptive learning rate, specific for each dimension of the gradient (with the aid of the second moment). In particular, dimensions that are steeper have a larger decay of the learning rate than dimensions that are closer to being flat, which considerably speeds up convergence. Although ADAM introduces three new parameters (one for each momentum, $\beta_1$ and $\beta_2$,  and one for numeric stability, $\epsilon$), it is quite robust to manual tuning, as the learning rate adjusts automatically as just explained. 

**Batch-size**. When using mini-batch GD, the batch size can help to find a good balance between stability and speed of the learning process. As already discussed above, a complete batch GD is completely stable, but it might take longer to converge and might not escape local minima that do not generalize well. On the other hand, stochastic GD is very noisy and might take longer to converge; it can escape bad local minima more easily, but given its large amount of noisy steps it might also fail to converge to good ones. Mini-batch GD is somewhat in the middle, and therefore adjusting the batch size can be beneficial to the GD algorithm. A good overview of GD methods is given [here](https://medium.com/@divakar_239/stochastic-vs-batch-gradient-descent-8820568eada1).   

**Activation Function**. Also the activation function for the hidden layers can be tuned. Even though the ReLU function is now widely used, it can still cause some issues. For example, it can suffer a phenomenon called "dying ReLU": if all the weights of a neuron lead to a negative value, the output of the ReLU function for this neuron becomes zero, making the neuron "die" and never be able to be active again during the next iterations of Gradient Descent. In some networks, you might even find several neurons dead simultaneosly. An alternative to the ReLU function is the [leaky relu](https://sefiks.com/2018/02/26/leaky-relu-as-an-neural-networks-activation-function/), which allows for small negative values. In this case, neurons may die during training, but have non-zero chance to be activated again in subsequent iterations of the training algorithm. ReLU functions tend to find more piece-wise linear functions, while functions such as the sigmoid and tanh allow for smoother shapes of the output function (or decision boundary, in case of classification). However, tanh and sigmoid suffer the [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem): with extreme values of their inputs, such functions become essentialy flat, leading to a derivative basically close to 0. In this way, backpropagation has no or little gradient to propagate back to the network, and no gradient is left for layers at lower levels. A different but related issue is the *exploding gradient problem*. More in general, neural networks suffer from unstable gradients, and different layers may learn at different speeds. Therefore, choosing a good activation function is crucial for the learning of the network.

**Number of Epochs and Early Stopping**. The number of epochs for GD methods is also crucial for learning. If the algorithm is run for too many epochs, it passes through the whole dataset too many times and it ends up learning too closely the training data, leading to poor generalization performance. However, a good number of epochs can lead to good generalization performance, while a too small number of epochs is not enough to make the model learn well, and it can cause underfitting. To decide how many epochs are good to let the algorithm generalize well, you can run the algorithm several times with different (increasing)  numbers of epochs, and evaluate each time its performance on a validation set. However, this method can be time-consuming, as you have to re-train the algorithm every time. A better strategy is given by **early stopping**: during training, you can assess the performance of the algorithm on the validation set at the end of each epoch. When the performance on this validation set stops improving, you can wait for a certain number of iterations (say, 10 or 20) to see whether the algorithm is actually starting to overfit; in that case, you can just stop the algorithm and use the estimates of the model obtained with the best epoch. An example of how early stopping works is given by this figure: 

<br>
<img src="./img/neural_networks/early_stopping.png" width="500"/>
<br>

The following figure shows the analogy between early stopping and $l_2$ regularization: 

<br>
<img src="./img/neural_networks/l2_early_stopping.png" width="400"/>
<br>

In practice, because the algorithm starts with weights initialized close to 0 (and that usually increase after each mini-batch update), early stopping is a way to find a simplified solution of the network; this is exactly the same that happens with $l_2$ regularization, where a penalty parameter must be tuned in order to lead to optimal generalization performance.  

**$l_1$ and $l_2$ regularization**. Another way to protect against overfitting is by obtaining simple solutions of the network (that is, setting its weights closer to 0). This is exactly what we have already seen for linear models: with $l_2$ regularization, all the weights are shrunken towards (but never equal to) 0; the effect is similar to the one of early stopping just described. With $l_1$ penalty, instead, we allow for *sparse* networks, where some of the connections between the neurons are equal to 0, and are dropped from the model. 


## 7. Other Architectures
There exists a [huge number of variants](https://www.asimovinstitute.org/neural-network-zoo/) of Neural Networks, each of which is devised for specific usage and context. Here, we are going to quickly talk about two among the most famous of these architectures: Convolutional Neural Networks (CNN) and Recurrent Neural Networks (RNN). 

### 7.1 Convolutional Neural Networks
CNN's are probably the reason number one why NN's have gained popularity again in recent years. They allow the machines to perform a task that is very basic for humans, but not so trivial for computers: image recognition. In particular, by using an analgoy with the [visual cortex](https://en.wikipedia.org/wiki/Visual_cortex) of the brain, they translate into a NN context the concept of *local receptive field*, with which the human brain reacts to local stimuli of a limited region of the visual field; when the neurons overlap such stimuli, they are able to recognize the object we are observing. 

In a similar fashion, CNN's try to process the various areas of a rectangular image locally, and subsequently -layer after layer- they try to combine the regions together, until they understand in the output layer what type of objects is represented in the figure. CNN's owe their name to the fact that they use *convolutions*. In this context, a convolution can be seen as a sort of matrix multiplication, slided across the rows of one of its inputs. In CNN's convolutions are computed for all the pixels of a figure by means of a *filter* or *convolutional kernel*, which is able to capture the local behaviour of the intensity of the pixels, and summarize it in a number. This operation is desirable for image recognition, as we want the method to be robust against rotations and shifting of the objects in the figure. The following figure summarizes the convolution operation: 

<br>
<img src="./img/neural_networks/convolution.png" width="400"/>
<br>

In images with colors, such operation is repeated (with different kernels) for each RGB channel of the image. After the convolutions have been computed, their result- a *feature map*- is passed through the first hidden layer of the network. Because CNN use the concepts of *receptive field* to capture the local behaviour of the previous layers, the first layers of a CNN (called convolutional layers) are also represented in 2D, so that it is possible to match the neuron of a layer with the one of the subsequent layer. The layers are then processed through other convolutions (or other types of operations, such as *pooling*). Finally, the last layers of the network are dense (fully-connected) layers (exactly like the ones observed in FNN) and the network terminates with an output layer, as usual. In general, CNN's are used for classification tasks (so that it is possible to recognize a category of object present in the picture). Because the task of CNN's is much more complicated than the ones of FNN's, their architecture is generally deeper than the architecture of a FNN. A more detailed introduction on the functioning of CNN's can be found in [this link](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53). 


<br>
<img src="./img/neural_networks/cnn.png" width="700"/>
<br>


### 7.2 Recurrent Neural Networks
FNN's allow the signal to travel through the network only in one direction, from input to output. RNN's have a similar structure of FNN's, but with an important difference: they introduce loops in the network, allowing the signal to travel also backward. This makes RNN's the ideal tool to model sequential data and time series, as they are likely to contain autocorrelations with time points of the past. In particular, when we observe inputs and outputs at different time points, the input of a layer at time point $t$ can use the output of the network at time point $t-1$ to model this type of dependencies. In practice, this is a way to take into account the evolution of a phenomenon over time. This makes RNN's a powerful forecasting tool. They are becoming widely tested and applied in fields such as stock market forecasting, and speech recognition (when a sequence of words must be recognized by the system). An issue of RNN's is that they tend to loose the information of time points very far in the past; this is know are "short memory" problem. Special types of RNN's architectures devised to overcome this issue are the LSTM (Long Short-Term Memory) and the GRU (Gater Reccurent Unit) networks. An introduction to RNN's is [in this link](https://www.geeksforgeeks.org/introduction-to-recurrent-neural-network/). 

<br>
<img src="./img/neural_networks/rnn.png" width="400"/>
<br>


## 8. Other Resources
* the [scikit-learn documentation](https://scikit-learn.org/stable/modules/neural_networks_supervised.html) on neural networks
* the [Deep Learning Book](https://www.deeplearningbook.org/) is freely available online; it contains a good and detailed overview of state-of-the-art Neural Networks (includig method for unsupervised learning and autencoders). The book is a bit technical, but it offers a nice review of linear algebra and other mathematical tools in the first part 
* in the Youtube channel of "3 Blue 1 Brown" there is a nice mini-series on Feed-Forward Neural Networks, explained with an example of digit recognition. The vision of such videos is highly recommended, as they are fun to watch and explain Neural Networks with amazing graphics. In particular, videos 2, 3, and 4 provide an accurate explanation of Gradient Descent and the Backpropagation algorithm. [Here](https://www.youtube.com/watch?v=aircAruvnKk&t=1008s) is the link to the first video of the series. 